In [1]:
import glob
import os
import pandas as pd

# Fungsi untuk import data ulasan perjalanan
def get_files(path):
    review = []
    abstractive = []
    kategori = []
    
    for i in range(4):
        for j in range(25):
            data = pd.read_excel('./data/tripadvisor/travel/' + str(path) + '/' + str(i+1) + '/' + str(j) + '.xlsx', usecols=['review', 'abstractive'], engine='openpyxl', header=0)
            data_length = len(data)
            data = data.astype(str)

            str_rev = ""
            for d in range(data_length):
                str_rev += " " + data['review'][d]
        
            review.append(str_rev)
            abstractive.append(data['abstractive'][0])
            kategori.append(path)
        
    return review, abstractive, kategori

In [2]:
# Proses import data untuk kategori hotel, restorant, dan tempat wisata
path = ['labeled-hotel','labeled-restaurant','labeled-attraction']
review = []
extractive = []
abstractive = []
kategori = []
for dt in path:
    r, a, k = get_files(dt)
    review.extend(r)
    abstractive.extend(a)
    kategori.extend(k)

data = pd.DataFrame(columns=['review','abstractive','kategori'])
data['review'] = review
data['abstractive'] = abstractive
data['kategori'] = kategori

In [3]:
# Contoh ulasan
data['review'][0]

' Nice place klo lagi nyari tempat staycation di tengah2 kota, makan siang di GI lalu makan malam di PP atau ketemuan ama temen di blok M ga masalah tinggal naik MRT. Ke kota tua lama? Tinggal naik Transjakarta karena @allseasonsjt cuma selemparan batu dari stasiun MRT, Commuter line, Transjakarta dan bahkan kereta bandara.  . . Yg hobby CFDan pas hari minggu pas banget deh ini hotelnya ðŸ˜Š  o iya klo mau dpt scenery view kek gw gini pilih yg deluxe city view; request kamar di lantai atas. Kebetulan kamar saya ada di lantai 11. Walaupun  untuk kamar mandinya hitungannya minimalis dan agak terlalu remang2 untuk ukuran keluarga yg membawa anak kecil. Tp overall cukup baik.  . .  Lokasi tengah kota , dekat stasiun sudirman bisa jalan kaki. Sky Lounge dengan pemandangan menarik dan cozzy. Setiap kamis malam ada live music. Makanan enak pas bumbu nya dan seperti masakan di rumah. Menu Andalan Sup Buntut yg empuk dan Buntut balado. Yummy. Minuman Coktail sesuai dan harga yang realistis. kam

In [4]:
import nltk
from nltk.corpus import stopwords
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import itertools
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize 
from langdetect import detect

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
stop_factory = stopwords.words('indonesian')

factory = StemmerFactory()
stemmer = factory.create_stemmer()

def first_process(text):                           
    text = re.sub(r'(\W)(?=\1)', '', text)              # menghapus tanda baca yang berlebihan
    text = re.sub(r'https?://\S+|www\.\S+', '', text)   # menghapus URL
    text = re.sub(r'[^\w\s]',' ', text)                 # menghapus tanda baca
    text = re.sub(r'  ',' ', text)                      # menghapus spasi yang berlebihan
    text = re.sub(r'[^\x00-\x7F]+',' ', text)           # menghapus non-ASCII
    text = text.strip()
    return text

# Fungsi normalisasi kalimat
def text_normalize(key_file, text):
    key_norm = pd.read_csv(key_file)
    text = ' '.join([key_norm[key_norm['slang'] == word]['formal'].values[0] if (key_norm['slang'] == word).any() else word for word in text.split()])
    text = str.lower(text)
    return text

# Fungsi menghapus stopword
def remove_stop_words(text):
    sw = stop_factory
    clean_words = []
    text = text.split()
    for word in text:
        if word not in sw:
            clean_words.append(word)
    return " ".join(clean_words)


def clean_process(text):
    text = str(text)
    text = text.lower()    
    text = first_process(text)
    text = text_normalize('./data/base/colloquial-indonesian-lexicon.csv',text)
    return text

In [14]:
from nltk.tokenize import sent_tokenize, word_tokenize
import string

# Proses tokenisasi dan menghapus kalimat kurang dari 1
def split_n_clean(data):
    cleaned_txt = []
    for i in range(len(data)):
        ctx = []
        st = re.split(r'(?<=[,.])(?<!\d.)\s', data['review'][i])
        for c in st:
            if (len(c.split())) > 1:
                ctx.append(clean_process(c))
        cleaned_txt.append(ctx)
        
    return cleaned_txt

In [15]:
st_cl = split_n_clean(data)

In [16]:
st_cl[0]

['nice place kalo lagi mencari tempat staycation di tengah-tengah kota',
 'makan siang di gi lalu makan malam di pp atau ketemuan sama teman di blok sama enggak masalah tinggal naik mrt',
 'ke kota tua lama tinggal naik transjakarta karena allseasonsjt cuma selemparan batu dari stasiun mrt',
 'transjakarta dan bahkan kereta bandara',
 'yang hobby cfdan pas hari minggu pas banget deh ini hotelnya o iya kalo mau dapat scenery view kayak gue begini pilih yang deluxe city view request kamar di lantai atas',
 'kebetulan kamar saya ada di lantai 11 walaupun untuk kamar mandinya hitungannya minimalis dan agak terlalu remang-remang untuk ukuran keluarga yang membawa anak kecil',
 'tapi overall cukup baik',
 'lokasi tengah kota',
 'dekat stasiun sudirman bisa jalan kaki',
 'sky lounge dengan pemandangan menarik dan cozzy',
 'setiap kamis malam ada live music',
 'makanan enak pas bumbu nya dan seperti masakan di rumah',
 'menu andalan sup buntut yang empuk dan buntut balado',
 'minuman coktail s

In [17]:
document = []
tmp_index = []
kategori = []
objek = []
tmp_review = []
index_file = 0
for i in range(len(data)):  
    document.extend(st_cl[i])
    index = 0
    if i != 0:
        if data['kategori'][i] != data['kategori'][i-1]:
            index_file = 0
            
    for t in range(len(st_cl[i])):
        tmp_index.append(index)
        kategori.append(data['kategori'][i])
        objek.append(index_file)
        index += 1
    index_file += 1

In [18]:
review_cln = []
for pst in st_cl:
    review_cln.append(' [SEP] '.join(pst) + '.')

In [19]:
data['review_cln'] = review_cln

In [20]:
data.to_csv('./data/tripadvisor/travel/sep_raw_review2.csv')

In [20]:
doc_pd = pd.DataFrame(columns=['index','objek','kategori','review'])
doc_pd['review'] = document
doc_pd['index'] = tmp_index
doc_pd['kategori'] = kategori
doc_pd['objek'] = objek

In [21]:
doc_pd[:50]

,index,objek,kategori,review
0,0,0,labeled-hotel,nice place kalo lagi mencari tempat staycation...
1,1,0,labeled-hotel,ke kota tua lama tinggal naik transjakarta kar...
2,2,0,labeled-hotel,yang hobby cfdan pas hari minggu pas banget de...
3,3,0,labeled-hotel,kebetulan kamar saya ada di lantai 11
4,4,0,labeled-hotel,walaupun untuk kamar mandinya hitungannya mini...
5,5,0,labeled-hotel,tapi overall cukup baik
6,6,0,labeled-hotel,lokasi tengah kota dekat stasiun sudirman bisa...
7,7,0,labeled-hotel,sky lounge dengan pemandangan menarik dan cozzy
8,8,0,labeled-hotel,setiap kamis malam ada live music
9,9,0,labeled-hotel,makanan enak pas bumbu nya dan seperti masakan...
